In [1]:
import gensim, logging
import pandas as pd
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/Users/mathurtx/tensorflow/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
read_csv = pd.read_csv('data/lastfm-dataset-1K/SongSequenceFor30MinutesSpaceDelimitedv2.csv/part-r-00000-75eb2cb8-ccc1-4118-9886-311798727607.csv', 
                       header=None)

In [3]:
trackIdName = pd.read_csv('/Users/mathurtx/Documents/machine_learning/data/lastfm-dataset-1K/trackIdTrackNamev2.csv/part-r-00000-a27872ab-ff1e-41e8-a846-a4af3fb0c702.csv', header=None, sep='|',error_bad_lines=False)

Skipping line 91094: expected 2 fields, saw 3

Skipping line 1257864: expected 2 fields, saw 4



In [4]:
sentences = read_csv[0].apply(lambda x: x.split(' '))

In [5]:
sentencesList = list(sentences)

In [6]:
filteredSentencesList = [x for x in sentencesList if len(x) > 1]

In [7]:
# train word2vec on the two sentences
model = gensim.models.Word2Vec(filteredSentencesList, size=200)

2016-12-16 08:47:25,801 : INFO : collecting all words and their counts
2016-12-16 08:47:25,802 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-12-16 08:47:25,820 : INFO : PROGRESS: at sentence #10000, processed 64047 words, keeping 9058 word types
2016-12-16 08:47:25,837 : INFO : PROGRESS: at sentence #20000, processed 116487 words, keeping 19175 word types
2016-12-16 08:47:25,867 : INFO : PROGRESS: at sentence #30000, processed 187406 words, keeping 29336 word types
2016-12-16 08:47:25,888 : INFO : PROGRESS: at sentence #40000, processed 244058 words, keeping 39085 word types
2016-12-16 08:47:25,912 : INFO : PROGRESS: at sentence #50000, processed 301737 words, keeping 48578 word types
2016-12-16 08:47:25,939 : INFO : PROGRESS: at sentence #60000, processed 359429 words, keeping 60812 word types
2016-12-16 08:47:25,962 : INFO : PROGRESS: at sentence #70000, processed 423365 words, keeping 68763 word types
2016-12-16 08:47:25,990 : INFO : PROGRESS: at se

In [8]:
trackIdName = trackIdName.rename(columns = {0:'track_id', 1: 'track_name'})

In [12]:
trackIds = list(trackIdName.loc[trackIdName['track_name'] == 'Neighborhood #3 (Power Out)']['track_id'])

In [15]:
trackIds

['9a6f09d5-b0b0-4ce7-8b83-c354bcbd347d',
 '07689d4c-540d-4518-9a41-52931b7f061f']

In [16]:
similar_items = model.most_similar('07689d4c-540d-4518-9a41-52931b7f061f')

In [17]:
similar_items

[('798112f7-7acd-4a2d-9436-baf93a7574f3', 0.9582799673080444),
 ('5551e780-656c-410f-91ff-03aed3af9b7a', 0.9525673389434814),
 ('d2e67c9e-3600-450c-aa69-cab51ff0c990', 0.941468358039856),
 ('e6d80ec5-85b3-4900-8e52-f83e9bf16852', 0.9325342178344727),
 ('fbf3c1c1-b5cf-41a1-8818-ccb3c7e231be', 0.9318362474441528),
 ('a1a4b6c4-d1c5-4a26-b155-75052d407c83', 0.9315561056137085),
 ('31973462-9cc7-4adb-a933-2442e625fbca', 0.9274295568466187),
 ('f874c752-65bc-4d50-ac7e-932243ae9f02', 0.9087802171707153),
 ('e58be0e6-3aec-4ee3-a3ee-d93073d0597a', 0.8932747840881348),
 ('1aa8c28b-728c-48e5-a0e3-ce511158a8a6', 0.8917827606201172)]

In [18]:
similar_items_list = [x[0] for x in similar_items]

In [19]:
similar_items_list_df = pd.DataFrame(similar_items_list)

In [20]:
similar_items_list_df = similar_items_list_df.rename(columns={0:'track_id'})

In [21]:
similar_items_list_df.head(5)

,track_id
0,798112f7-7acd-4a2d-9436-baf93a7574f3
1,5551e780-656c-410f-91ff-03aed3af9b7a
2,d2e67c9e-3600-450c-aa69-cab51ff0c990
3,e6d80ec5-85b3-4900-8e52-f83e9bf16852
4,fbf3c1c1-b5cf-41a1-8818-ccb3c7e231be


In [22]:
similar_items_list_name = pd.merge(similar_items_list_df, trackIdName, how='left', left_on='track_id', right_on='track_id')

In [23]:
similar_items_list_name

,track_id,track_name
0,798112f7-7acd-4a2d-9436-baf93a7574f3,Neighborhood #2 (Laika)
1,5551e780-656c-410f-91ff-03aed3af9b7a,Crown Of Love
2,d2e67c9e-3600-450c-aa69-cab51ff0c990,Une Année Sans Lumière
3,e6d80ec5-85b3-4900-8e52-f83e9bf16852,Neighborhood #1 (Tunnels)
4,fbf3c1c1-b5cf-41a1-8818-ccb3c7e231be,Neighborhood #4 (7 Kettles)
5,a1a4b6c4-d1c5-4a26-b155-75052d407c83,Wake Up
6,31973462-9cc7-4adb-a933-2442e625fbca,Haiti
7,f874c752-65bc-4d50-ac7e-932243ae9f02,Rebellion (Lies)
8,e58be0e6-3aec-4ee3-a3ee-d93073d0597a,Neighborhood #2 (Laïka)
9,1aa8c28b-728c-48e5-a0e3-ce511158a8a6,In The Back Seat


In [3]:
model = gensim.models.Word2Vec.load(fname='data/lastfm-dataset-1K/songsWord2Vec.csv')

2016-12-16 14:57:15,777 : INFO : loading Word2Vec object from data/lastfm-dataset-1K/songsWord2Vec.csv


UnpicklingError: could not find MARK

In [8]:
model.save_word2vec_format(fname='data/lastfm-dataset-1K/songsWord2Vec.csv')

2016-12-16 07:24:55,255 : INFO : storing 416167x200 projection weights into data/lastfm-dataset-1K/songsWord2Vec.csv
